In [1]:
import tensorflow as tf 
import pandas as pd 
from tensorflow.keras.layers import Dropout, Conv1D, MaxPool1D, Embedding,SimpleRNN, LSTM, Input, Concatenate 
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dense, Flatten 
import numpy as np 
import string 
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing import sequence 
from sklearn.datasets import fetch_20newsgroups 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
import nltk 
nltk.download('punkt') 
from nltk.corpus import stopwords 
import re    
from nltk.tokenize import word_tokenize   
from tensorflow.keras.utils import to_categorical, plot_model 
from tensorflow.keras.callbacks import EarlyStopping 
from sklearn.model_selection import train_test_split 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [3]:
url = '/content/drive/MyDrive/Classeur1.csv'

In [4]:
df = pd.read_csv(url, sep = ';')
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
d={'address':1,'code':0,'contact':0,'other':0}
df['label']=df['label'].map(d)

In [6]:
X = df.text
y = df.label

In [7]:
x=list()
Y=list()
for i in range(X.shape[0]):
    x.append(X[i])

In [8]:
Y=list()
for i in range(X.shape[0]):
    Y.append(y[i])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x,Y,test_size=0.2)

In [11]:
import datasets
dd = datasets.DatasetDict({"train":(X_train,y_train),"test":(X_test,y_test)})

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 16.8 MB/s 
     |████████████████████████████████| 7.6 MB 48.1 MB/s 


In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
tokenized_data = tokenizer(dd["train"][0], return_tensors="np", padding=True)

In [15]:
labels = np.array(dd["train"][1])

In [16]:
labels

array([1, 0, 1, ..., 0, 0, 1])

In [17]:
from transformers import TFAutoModelForSequenceClassification 
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5), loss=loss, metrics=metric)

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(tokenized_data, labels)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


337/337 [==============================] - 207s 614ms/step - loss: 0.0391 - sparse_categorical_accuracy: 0.9871


In [35]:
test = 'cliente number : vb 459472'
test=tokenizer.encode(test,truncation=True,padding=True,return_tensors="tf")                                                                                      

In [27]:
prediction = model.predict(test)
prediction_logits = prediction[0]
prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()

1/1 [==============================] - 0s 168ms/step


In [28]:
prediction_probs

array([[9.9974185e-01, 2.5816564e-04]], dtype=float32)

In [49]:
model.save_weights('/content/drive/MyDrive/Projet_BI/weights.hdf5')

In [57]:
model.save_weights('/content/drive/MyDrive/Projet_BI/weights.tf')

In [59]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Projet_BI/weights.tf.index')

OSError: ignored

In [37]:
prediction = model1(test)
prediction_logits = prediction[0]
prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()

TypeError: ignored